In [1]:
import os
import MeCab
import pickle
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# ---------------------------------------------------------
# データの作成
# ---------------------------------------------------------
def create_data(author_name, txt_files):
  txt_dir = 'txt_' + author_name
  book_list = []
  content_list = []

  # ファイルの読み込みとコンテンツの保存
  for file_name in txt_files:
    file_path = os.path.join(txt_dir, file_name)
    with open(file_path, 'r', encoding='shift-jis') as file:
      book_list.append(file.readline().strip())                   # 書籍名を取得
      content_list.append(tokenize(sanitize(file.read())))        # 不要な部分を削除したコンテンツのトークンを取得

  return book_list, content_list

# ---------------------------------------------------------
# コンテンツの不要な部分を除外
# ---------------------------------------------------------
def sanitize(text):
  operations = [
    lambda text: re.split(r'\-{5,}', text)[2],
    lambda text: re.split(r'底本：', text)[0],
    lambda text: re.sub(r'《.+?》', '', text),
    lambda text: re.sub(r'［＃.+?］', '', text),
    lambda text: text.strip()
  ]

  for operation in operations:
    try:
      text = operation(text)
    except Exception as e:
      pass

  return text

# ---------------------------------------------------------
# 形態素解析
# ---------------------------------------------------------
def tokenize(text):
  tagger = MeCab.Tagger("")
  node = tagger.parseToNode(text)
  tokens = []

  while node is not None:
    part = node.feature.split(",")[0]
    if part in ["名詞", "動詞", "形容詞"]:
      tokens.append(node.surface)
    node = node.next

  return tokens

In [3]:
# ---------------------------------------------------------
# メイン
# ---------------------------------------------------------
with open("book_data.pkl", "rb") as f:          # rb = read-binary
    data = pickle.load(f)

author_list = data["author_list"]
file_list   = data["file_list"]

tagged_data = []

# データの作成
for i in range(len(author_list)):
  book_list, content_list = create_data(author_list[i], file_list[i])
  for j in range(len(book_list)):
    tag = f'{author_list[i]}：{book_list[j]}'
    tagged_data.append(TaggedDocument(words=content_list[j], tags=[tag]))

# Doc2Vecモデルの作成
model = Doc2Vec(vector_size=50, min_count=2, epochs=50)
model.build_vocab(tagged_data)
model.train(tagged_data, total_examples=model.corpus_count, epochs=model.epochs)

# モデルの保存
model.save("d2v.model")

In [4]:
# モデルのロード
model = Doc2Vec.load("d2v.model")

# 新しいテキストのトークン化
new_text = '青びかる天弧のはてに、きらゝかに町はうかびて、六月のたつきのみちは、いまやはた尽きはてにけり。いさゝかの書籍とセロを、思ふまゝ'
new_text_tokens = tokenize(new_text)
new_vector = model.infer_vector(new_text_tokens)

# 類似度の計算
similar_docs = model.dv.most_similar([new_vector], topn=10)

# 結果の表示
for label, similarity in similar_docs:
  print(f"{label}：{similarity}")

宮沢 賢治：〔青びかる天弧のはてに〕：0.8948869705200195
宮沢 賢治：青柳教諭を送る：0.8413702249526978
北原 白秋：風見：0.8381561040878296
宮沢 賢治：〔あくたうかべる朝の水〕：0.8112357258796692
宮沢 賢治：〔雨ニモマケズ〕：0.7816504240036011
北原 白秋：影：0.7568415999412537
芥川 竜之介：芥川龍之介歌集：0.6999874711036682
北原 白秋：浅草哀歌：0.6797680258750916
芥川 竜之介：愛読書の印象：0.6123363375663757
北原 白秋：お月さまいくつ：0.6065266132354736


In [5]:
# モデルのロード
model = Doc2Vec.load("d2v.model")

# 新しいテキストのトークン化
new_text = '青びかる天弧のはてに、きらゝかに町はうかびて、六月のたつきのみちは、いまやはた尽きはてにけり。いさゝかの書籍とセロを、思ふまゝ'
new_text_tokens = tokenize(new_text)
new_vector = model.infer_vector(new_text_tokens)

# すべてのドキュメントベクトルを取得
doc_vectors = np.array([model.dv[i] for i in range(len(model.dv))])
doc_labels = [model.dv.index_to_key[i] for i in range(len(model.dv))]

# コサイン類似度の計算
new_vector = new_vector.reshape(1, -1)
cosine_similarities = cosine_similarity(new_vector, doc_vectors).flatten()
similar_indices = cosine_similarities.argsort()[-10:][::-1]

# 結果の表示
for idx in similar_indices:
  print(f"{doc_labels[idx]}：{cosine_similarities[idx]}")

宮沢 賢治：〔青びかる天弧のはてに〕：0.8948871493339539
宮沢 賢治：青柳教諭を送る：0.8413703441619873
北原 白秋：風見：0.8381561040878296
宮沢 賢治：〔あくたうかべる朝の水〕：0.811235785484314
宮沢 賢治：〔雨ニモマケズ〕：0.7816504240036011
北原 白秋：影：0.7568415999412537
芥川 竜之介：芥川龍之介歌集：0.6999874711036682
北原 白秋：浅草哀歌：0.6797680854797363
芥川 竜之介：愛読書の印象：0.6123364567756653
北原 白秋：お月さまいくつ：0.6065266728401184
